In [1]:
"""
Link daily Fire Radiative Power (FRP) retrievals to daily fire perimeters
Author: maxwell.cook@colorado.edu
"""

import os, time, glob, gc
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pytz

from datetime import datetime
from shapely.geometry import box
from shapely.geometry import Polygon, MultiPolygon

import warnings
warnings.filterwarnings("ignore") # suppresses annoying geopandas warning

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [2]:
def create_bounds(gdf, buffer=None, by_bounds=True):
    """ Calculate a bounding rectangle for a given geometry and buffer """
    if by_bounds is True:
        geom = gdf.geometry.apply(lambda geo: box(*geo.bounds))
        if buffer is not None:
            geom = geom.buffer(buffer)
        # Apply the new geometry
        gdf_ = gdf.copy()
        gdf_.geometry = geom.geometry.apply(
            lambda geo: Polygon(geo) if geo.geom_type == 'Polygon' else MultiPolygon([geo]))
        return gdf_
    elif by_bounds is False:
        if buffer is not None:
            geom = gdf.geometry.buffer(buffer)
        else:
            gdf_ = gdf.copy()
            gdf_.geometry = gdf_.geometry.buffer(buffer)
            return gdf_    

 
def convert_datetime(acq_date, acq_time):
    """ Function to convert ACQ_DATE and ACQ_TIME to a datetime object in UTC """
    # Ensure ACQ_TIME is in HHMM format
    acq_time = str(acq_time) # force to string
    if len(acq_time) == 3:
        acq_time = '0' + acq_time
    elif len(acq_time) == 2:
        acq_time = '00' + acq_time
    elif len(acq_time) == 1:
        acq_time = '000' + acq_time

    acq_date_str = acq_date.strftime('%Y-%m-%d')
    dt = datetime.strptime(acq_date_str + acq_time, '%Y-%m-%d%H%M')
    dt_utc = pytz.utc.localize(dt)  # Localize the datetime object to UTC
    return dt_utc


print("Functions read !")

Functions read !


In [3]:
# Load the fire data.

In [4]:
aspen_events_fp = os.path.join(projdir,'data/spatial/mod/FIRED/fired-events_west_2018_to_2024_aspen.gpkg')
aspen_events = gpd.read_file(aspen_events_fp)
aspen_events.columns

Index(['id', 'ig_date', 'ig_day', 'ig_month', 'ig_year', 'last_date',
       'event_dur', 'tot_pix', 'tot_ar_km2', 'fsr_px_dy', 'fsr_km2_dy',
       'mx_grw_px', 'mx_grw_km2', 'mx_grw_dte', 'ig_utm_x', 'ig_utm_y',
       'na_l3name', 'pct_aspen', 'geometry'],
      dtype='object')

In [5]:
# Create a fire perimeter buffer to extract FRP observations within 1km of bounds.

In [6]:
bounds = create_bounds(aspen_events, by_bounds=True, buffer=1000)
bounds = bounds.set_crs(proj, allow_override=True) # ensure correct crs
bounds = bounds[['id','ig_date','last_date','geometry']] 
bounds.head()

,id,ig_date,last_date,geometry
0,282,2021-08-05,2021-08-05,"POLYGON ((-2086715.084 2356782.795, -2087666.8..."
1,492,2018-08-09,2018-08-13,"POLYGON ((-2099220.550 2280974.827, -2105343.3..."
2,1796,2021-10-28,2021-11-04,"POLYGON ((-1796487.029 1997450.102, -1799267.9..."
3,2207,2021-10-30,2021-10-30,"POLYGON ((-1822312.155 1967211.915, -1823260.8..."
4,2314,2021-10-29,2021-10-29,"POLYGON ((-1758198.372 1945155.099, -1759124.2..."


In [7]:
# Load the Active Fire Detection (AFD) data

AFDs from MODIS Collection 6.1 (1km) and the Suomi National Polar-Orbiting Partnership (VIIRS S-NPP 375m) were obtained from the NASA FIRMS (https://firms.modaps.eosdis.nasa.gov/download/) between 2018-2023 in the western US. The S-NPP AFDs include an archive shapefile and a NRT shapefile for Sept 2022-->

In [8]:
# Gather the archive and NRT S-NPP AFDs.
snpp_fp = os.path.join(projdir,'data/spatial/raw/NASA-FIRMS/DL_FIRE_SV-C2_476784/')
vects = glob.glob(snpp_fp+"*.shp")
print([os.path.basename(v) for v in vects])

['fire_nrt_SV-C2_476784.shp', 'fire_archive_SV-C2_476784.shp']


In [9]:
# Merge the NRT and archive vintages
archive = gpd.read_file([v for v in vects if "archive" in v][0]).to_crs(proj)
nrt = gpd.read_file([v for v in vects if "nrt" in v][0]).to_crs(proj)
snpp = pd.concat([archive, nrt], ignore_index=True)
snpp.head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,36.622398,-119.958420,319.25,0.45,0.63,2018-01-01,0854,N,VIIRS,n,1,276.46,1.93,N,0.0,POINT (-2100437.814 1774627.392)
1,48.166241,-102.698212,323.92,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,244.01,3.01,N,0.0,POINT (-502299.189 2814197.682)
2,48.152390,-102.697792,325.34,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,247.16,2.07,N,0.0,POINT (-502375.241 2812672.753)
3,47.879730,-102.706543,320.74,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,245.34,3.00,N,0.0,POINT (-505150.481 2782728.231)
4,44.249660,-104.516678,327.49,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,251.79,5.84,N,0.0,POINT (-677212.174 2392519.636)


In [10]:
# Set up a dictionary to store both MODIS and VIIRS AFDs
modis_fp = os.path.join(projdir,'data/spatial/raw/NASA-FIRMS/DL_FIRE_M-C61_476781/')

# Store these in a dictionary
afds = {
    "MOD61": gpd.read_file(modis_fp).to_crs(proj),
    "SNPP": snpp
}

afds['MOD61'].head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,44.3762,-119.1190,312.5,1.2,1.1,2018-01-01,0608,Terra,MODIS,85,6.03,269.1,21.9,N,0,POINT (-1819292.701 2598646.798)
1,32.1679,-107.7655,322.3,1.1,1.0,2018-01-01,1809,Terra,MODIS,77,6.03,289.1,19.0,D,0,POINT (-1101321.656 1078752.715)
2,32.1643,-107.7433,318.4,1.1,1.0,2018-01-01,1809,Terra,MODIS,74,6.03,287.9,14.9,D,0,POINT (-1099303.652 1078097.526)
3,31.7188,-102.0085,303.8,1.4,1.2,2018-01-01,1944,Aqua,MODIS,57,6.03,279.4,12.6,D,0,POINT (-566656.554 978378.293)
4,38.8238,-122.6829,302.0,2.7,1.6,2018-01-01,1946,Terra,MODIS,50,6.03,280.2,26.9,D,0,POINT (-2264952.497 2075236.763)


In [11]:
# Extract the AFDs within fire bounds.

In [12]:
afd_events = {} # to store the filtered data
# Loop through sensors (MODIS and VIIRS)
for satellite, afd in afds.items():
    print(f"Processing: {satellite}")
    
    afd['VID'] = afd.index # add a unique ID
    
    # Remove low confidence observations
    try:
        afd = afd[afd['CONFIDENCE'] != 'l']
    except KeyError as e:
        print(f"KeyError: {e}")

    if afd.crs != bounds.crs:
        afd = afd.to_crs(bounds.crs)  # ensure the crs matches

    # Extract within fire bounds
    afd_ = gpd.sjoin(afd, bounds, how='inner', predicate='within')
    afd_.drop(columns=['index_right'], inplace=True)
    
    # Tidy the date columns for matching
    afd_['ACQ_DATE'] = pd.to_datetime(afd_['ACQ_DATE'])
    afd_['ACQ_MONTH'] = afd_['ACQ_DATE'].dt.month.astype(int)
    afd_['ACQ_YEAR'] = afd_['ACQ_DATE'].dt.year.astype(int)
    
    # Create a datetime object
    afd_.loc[:, 'ACQ_DATETIME'] = afd_.apply(lambda row: convert_datetime(row['ACQ_DATE'], row['ACQ_TIME']), axis=1)

    # Filter for matching dates
    afd_f = afd_[
        (afd_['ACQ_YEAR'] >= afd_['ig_date'].dt.year.astype(int)) & 
        (afd_['ACQ_MONTH'] >= afd_['ig_date'].dt.month.astype(int)) &
        (afd_['ACQ_YEAR'] <= afd_['last_date'].dt.year.astype(int)) &
        (afd_['ACQ_MONTH'] <= afd_['last_date'].dt.month.astype(int))
    ]

    # Drop any duplicates
    afd_f = afd_f.drop_duplicates(subset='VID', keep='first')
    afd_f.drop(columns=['VID'], inplace=True)
    
    print(f"\tThere are [{len(afd_f)}] high or nominal confidence detections within fire bounds.")

    # Overwrite the dictionary items
    afd_events[satellite] = afd_f

    # Save the AFD observation in fire bounds
    out_fp = os.path.join(projdir,f'data/spatial/mod/AFD/{satellite}-afd_fired-aspen_2018_to_2023.gpkg')
    afd_f.to_file(out_fp)
    print(f'Saved to {out_fp}\n')
    
    del afd, afd_, afd_f
    gc.collect()

print("\nProcessing complete !")

Processing: MOD61
	There are [28707] high or nominal confidence detections within fire bounds.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/MOD61-afd_fired-aspen_2018_to_2023.gpkg

Processing: SNPP
	There are [118561] high or nominal confidence detections within fire bounds.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/SNPP-afd_fired-aspen_2018_to_2023.gpkg


Processing complete !


In [13]:
afd_events['MOD61'].columns

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'geometry', 'id', 'ig_date',
       'last_date', 'ACQ_MONTH', 'ACQ_YEAR', 'ACQ_DATETIME'],
      dtype='object')

In [14]:
# Calculate daily FRP statistics

In [15]:
datetime_frp = {} # to store aggregated FRP by datetime
dob_frp = {} # by day of burn
for satellite, afd in afd_events.items():
    # Group by fire ID and calculate datetime 99th percentile FRP
    dt_frp = afd.groupby(['id', 'DAYNIGHT', 'ACQ_DATETIME'])['FRP'].quantile(0.99).reset_index(name='FRP99_DT')
    
    # Add a date object
    dt_frp['ACQ_DATE'] = dt_frp['ACQ_DATETIME'].dt.date

    def calc_frp_percentiles(group):
        """ Function to calculate day / night statistics """
        return pd.Series({
            'FRP_MN': group['FRP99_DT'].mean(), 
            'FRP_SD': group['FRP99_DT'].std(),
            'FRP_90': np.percentile(group['FRP99_DT'], 90),
            'FRP_95': np.percentile(group['FRP99_DT'], 95),
            'FRP_99': np.percentile(group['FRP99_DT'], 99)
        })

    # Calculate the daily FRP statistics by day/night
    date_frp = dt_frp.groupby(['id', 'ACQ_DATE', 'DAYNIGHT']).apply(calc_frp_percentiles).reset_index()
    date_frp['FRP_SD'].fillna(0, inplace=True) # NaN to 0 where there is only one observation
    
    # append to the dictionary
    datetime_frp[satellite] = dt_frp 
    dob_frp[satellite] = date_frp 
    
print("Day/night FRP statistics calculated !")

Day/night FRP statistics calculated !


In [16]:
datetime_frp['MOD61'].head()

,id,DAYNIGHT,ACQ_DATETIME,FRP99_DT,ACQ_DATE
0,492,N,2018-08-10 05:38:00+00:00,69.649,2018-08-10
1,492,N,2018-08-10 09:50:00+00:00,30.448,2018-08-10
2,2544,D,2018-09-05 18:15:00+00:00,933.770,2018-09-05
3,2544,D,2018-09-06 18:58:00+00:00,38.087,2018-09-06
4,2544,D,2018-09-06 20:36:00+00:00,263.746,2018-09-06


In [17]:
dob_frp['MOD61'].head()

,id,ACQ_DATE,DAYNIGHT,FRP_MN,FRP_SD,FRP_90,FRP_95,FRP_99
0,492,2018-08-10,N,50.0485,27.719293,65.7289,67.68895,69.25699
1,2544,2018-09-05,D,933.7700,0.000000,933.7700,933.77000,933.77000
2,2544,2018-09-05,N,183.5050,141.760767,263.6970,273.72100,281.74020
3,2544,2018-09-06,D,150.9165,159.565009,241.1801,252.46305,261.48941
4,2544,2018-09-07,D,26.9000,0.000000,26.9000,26.90000,26.90000


In [ ]:
# Aggregate to the DOB from the daily perimeters

In [18]:
# Load the FIRED perimeters (2012-2023)
daily_fp = os.path.join(maindir,'aspen-fire/Aim2/data/spatial/raw/FIRED/fired-daily_west_2012_to_2023.gpkg')
daily = gpd.read_file(daily_fp)
print(f"There are [{len(daily)}] daily perimeters (2012-2023).")

There are [70932] daily perimeters (2012-2023).


In [19]:
# Filter to aspen fires (2018-2023)
daily = daily[daily['id'].isin(aspen_events['id'].unique())]
daily = daily.to_crs(proj) # ensure albers projection
print(f"There are [{len(daily)}] daily perimeters for aspen fires (2018-2023).")

There are [3387] daily perimeters for aspen fires (2018-2023).


In [20]:
daily.columns

Index(['did', 'date', 'id', 'ig_date', 'ig_day', 'ig_month', 'ig_year',
       'last_date', 'event_dur', 'tot_pix', 'tot_ar_km2', 'fsr_px_dy',
       'fsr_km2_dy', 'mx_grw_px', 'mx_grw_km2', 'mx_grw_dte', 'ig_utm_x',
       'ig_utm_y', 'na_l3name', 'geometry'],
      dtype='object')

In [22]:
# Clean up the table for joining
daily_ = daily[['id','did','date']]
daily_.rename(columns={'date': 'burn_date'}, inplace=True)
daily_['id'] = daily_['id'].astype(str) # ensure id is a string for joining
daily_['burn_date'] = pd.to_datetime(daily_['burn_date']).dt.date # ensure date type
daily_ = daily_.sort_values(by=['id','burn_date'])
daily_.head()

,id,did,burn_date
61821,1796,df0679d5c4e0c44b0f5b36547175ce89,2021-10-28
43422,1796,9d0c0e530e80ee93b02752d9ab868c7f,2021-10-29
35122,1796,7f26e46c07eb94125b29ecd4a21926a7,2021-10-30
11416,1796,292bf2b5180da458108872171f792142,2021-10-31
35021,1796,7ee31b1beb043ddb3e3984fa3c74f2a3,2021-11-02


In [23]:
len(daily_['id'].unique())

715

In [24]:
len(dob_frp['MOD61']['id'].unique())

212

In [ ]:
# Merge the daily burn perimeters with daily FRP 

In [25]:
daynight_frp = {} # to store day / night FRP
for satellite, afd in dob_frp.items():
    # Make sure the data types match
    afd['id'] = afd['id'].astype(str)
    afd['ACQ_DATE'] = pd.to_datetime(afd['ACQ_DATE']).dt.date
    
    # Merge the daily perimeters with the AFD
    frp_events = pd.merge(daily_, afd, left_on=['id','burn_date'], right_on=['id','ACQ_DATE'], how='left')
    
    # Drop out fires with no AFD
    frp_events = frp_events.dropna(subset=['ACQ_DATE'])
    
    daynight_frp[satellite] = frp_events # append to new dictionary
    
print("Joined DOB to daily FRP !")

Joined DOB to daily FRP !


In [26]:
daynight_frp['MOD61'].columns

Index(['id', 'did', 'burn_date', 'ACQ_DATE', 'DAYNIGHT', 'FRP_MN', 'FRP_SD',
       'FRP_90', 'FRP_95', 'FRP_99'],
      dtype='object')

In [27]:
len(daynight_frp['MOD61'])

1535

In [28]:
len(daynight_frp['MOD61']['id'].unique())

155

In [34]:
daynight_frp['MOD61'][['id','burn_date','ACQ_DATE','DAYNIGHT','FRP_95']].head(10)

,id,burn_date,ACQ_DATE,DAYNIGHT,FRP_95
10,189371,2018-06-28,2018-06-28,D,2175.06680
11,189371,2018-06-28,2018-06-28,N,2355.28295
12,189371,2018-06-29,2018-06-29,D,186.42155
13,189371,2018-06-29,2018-06-29,N,1271.31255
14,189371,2018-06-30,2018-06-30,D,103.69200
15,189371,2018-06-30,2018-06-30,N,185.21490
16,189371,2018-07-01,2018-07-01,D,41.29880
17,189371,2018-07-01,2018-07-01,N,48.76840
18,189371,2018-07-02,2018-07-02,D,53.79200
19,189371,2018-07-02,2018-07-02,N,16.18560


In [ ]:
# Pivot and merge the LANDFIRE EVT

In [35]:
# Load the EVT summary
daily_evt_fp = os.path.join(projdir, 'data/tabular/mod/EVT/fired-daily_west_2018_to_2023-EVT.csv')
daily_evt = pd.read_csv(daily_evt_fp)
daily_evt.drop(columns=['Unnamed: 0', 'id'], inplace=True)
daily_evt.head()

,did,evt,count,total_pixels,pct_cover,EVT_NAME,EVT_PHYS,EVT_GP_N,EVT_CLASS
0,0003d89c00208d55887fec4a95f58ac8,7017,2,5026,0.039793,Columbia Plateau Western Juniper Woodland and ...,Conifer,Juniper Woodland and Savanna,Open tree canopy
1,0003d89c00208d55887fec4a95f58ac8,7080,4,5026,0.079586,Inter-Mountain Basins Big Sagebrush Shrubland,Shrubland,Big Sagebrush Shrubland and Steppe,Shrubland
2,0003d89c00208d55887fec4a95f58ac8,7123,5,5026,0.099483,Columbia Plateau Steppe and Grassland,Grassland,Grassland and Steppe,Herbaceous-shrub-steppe
3,0003d89c00208d55887fec4a95f58ac8,7127,5,5026,0.099483,Inter-Mountain Basins Semi-Desert Shrub-Steppe,Shrubland,Desert Scrub,Shrubland
4,0003d89c00208d55887fec4a95f58ac8,7296,36,5026,0.716275,Developed-Low Intensity,Developed-Low Intensity,Developed-Low Intensity,No Dominant Lifeform


In [ ]:
# Pivot longer to create EVT percent cover as columns

In [ ]:
gc.collect()